In [2]:
# Importing required libraries.
import numpy as np
import pandas as pd
import os
import glob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import operator
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [7]:
# func to save and load file to reduce processing time.
import pickle
def save(word_list, name):
    with open(name, 'wb') as fp:
        pickle.dump(word_list, fp, protocol=pickle.HIGHEST_PROTOCOL)

# func to load:
def load(name):
    with open(name, 'rb') as fp:
        data = pickle.load(fp)
        return data

In [3]:
# Loading files 

# 'List' or 'target' will contain the output of our data.
# path1 is the path in which required files are stored. 
lst = []
path1 = 'C:\\Users\\Archit\\Desktop\\new\\20_newsgroups_train'
for root, dirs, files in os.walk(path1):
    for name in dirs:
        lst.append(name)
target = lst
print(len(target))

# loading stop_words and punctuations:
# We have to remove stop_words and punctuations from email to reduce the unneccessary words.
stop_words = []
corpus_path = 'C:\\Users\\Archit\\Desktop\\new\\stopwords_en.txt'  # path where stop_words are stored.
review_file = open(corpus_path,"r+")#.read()
    #print(review_file)
for line in review_file:
    stop_words+=review_file.readlines()
    
# 'punct' will contain punctuations. 
punct = string.punctuation
punct = list(punct)

20


In [6]:
# If stop_words contains '\n' then, UNCOMMENT this cell to remove '\n' from all stop_words.
# stop_words = [ w.replace("\n", "") for w in stop_words]
# print(stop_words,end=" ")

In [8]:
# Now, all_words will contain all the words from each file.
# this may take a while.
all_words = []
for name in lst:    # loop over each folder.
    path = "C:\\Users\\Archit\\Desktop\\new\\20_newsgroups_train\\" + name   # path now becomes path_folder + name of folder.
    for file in glob.glob(os.path.join(path, '*')):
        file_o = open(file,'r').read()                                 
        #print(file_o)
        words = file_o.split()
        word_len = len(words)
        for w in words:                              
            if w!='Date:' or w!='date:':        # removing header when 'date' is encountered.
                words.remove(w)
            elif w=='Date:' or w=='date:':
                break
        k = 0.03*(word_len)                    # removing starting 3% words from each file after 'date' encountered.
        k=int(k)
        words = words[k:]                                                   # removing header.
        words = [x.lower() for x in words]                                   # converting to lower_case.
        all_words += words
len(all_words)

2296275

In [9]:
# creating dictionary of all the words with key as word_name and count of that word as value.
from collections import Counter      # inbuilt function.
dictionary = Counter(all_words)      # dictionary will store (word_name, word_cnt) as key and value resp.
key = dictionary.keys()
len(dictionary)

217764

In [10]:
# sorting dictionary in descending order based on values.
sorted_dict = sorted(dictionary.items(), key=operator.itemgetter(1), reverse=True)
len(sorted_dict)

217764

In [ ]:
print(sorted_dict[:500],end='')

In [12]:
# Initialising features as NULL. 
# Iterate over each word in sorted dictionary (descending order) and check if it's not an stop_word or string_punc then add this word in feature_list.
features=[]
for name,cnt in sorted_dict:
    if name not in stop_words and name not in punct and name.isdigit()==False and len(name)>1 and len(name)<=20:     # for cleaning purpose.
        features.append(name)
features = features[30:]                 # removing topmost 30 words(as they're too common so they're of no use).
print(len(features),end=" ")

183042 

In [14]:
# cleaning the features_list to remove unwanted words.

remove_list_1 = ['called','__________________|always','whatever)','us).','________________________________________________________________________________steven', 'o---------------------------+======================================o','ace||','bs>','at?','return;', 'xfree86?', 'perror("");', 'please...', '3do','com2','dc,','from:','files?', 'ave.,', 'sandvik@newton.apple.com', 'x.b','hmmm,',  "doesn't,", 'b.c.,', 'sandvik)','like?', 'des?','halat@bear.com', 'asala/sdpa/arf', 'to/from',  'rider)','usa)',"isn't,",  'a&m', 'ny,', 'game?', 'well...','to/from', 'eggs', 'anyway).', 'lynn', 'er,', 'wave/qwk', 'des?', 'ch>', 'roussel]','po5.andrew.cmu.edu', 'ed.,','rm.', 'dc,', 'eli', 'space?', 'bontchev@fbihh.informatik.uni-hamburg.de','hp-ux', 'names.', 'machine)', 'r&d',"doesn't,", 'halat@bear.com', 'sandvik@newton.apple.com', 'sandvik)', 'congress..."', 'ed.green@east.sun.com', 'her!"', 'ppm', 'thf2@kimbark.uchicago.edu', 'jpeg,', 'post)',  'number?', 'tk>',"isn't,",'children?',  'snichols@adobe.com',  'horne)', 'fred.mccall@dseg.ti.com',  'nntp@acsu.buffalo.edu',  'z1dan@exnet.iastate.edu', 'cb360t',  'mrs.', 'comp.windows.x',  'too).', 'luck!', 'kmr4@po.cwru.edu', 'ryan)',   'xt,', 'hmmm...','get?',"other's",'on...', 'rll',  '50mhz', 'uupcb', 'amd','king)', 'bj-200', 'ie.', 'quote:', 'group)',  'obo.','is).', 'them)', 'again?', 'jmd@handheld.com', 'atf/fbi', 'viking@iastate.edu', 'postmaster@ozonehole.com', 'nut', 'iici', 'thu', 'below,',  'wont',  'line?', 'eczcaw@mips.nott.ac.uk', 'werth!',  'hence,', 'site,', 'whatsoever', 'chip?', '512k','bible?', 'nt,', 'ssto', 'at,','correct?', 'js>', 'ps/2', 'time."', 'available?','he?', 'baalke@kelvin.jpl.nasa.gov', 'jb>', 'cover)','had.','geb@cs.pitt.edu', 'nsmca@acad3.alaska.edu', 'jacked', 'kratz', 'uh,', 'here)','w4wg','baalke@kelvin.jpl.nasa.gov', 'deleted...]', 'name)', 'question...', 'pm)','call', 'too)', "ol'", 'etc.?', 'that).','is!', 'issue?', 'anything?', 'time)', "max>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'",  'time...', 'signature-----', 'time...','call', 'ok.', 'e.g.', 'can.','again...', 'me).','else?', 'me;', 'great','we?','more)', 'do)', 'in)','it.)', 'them."','me)', 'are?','is)','cars?','yalcin]','them).', 'arms,-never--never--never!"', 'one).','it),','etc?','myrto)','osf/motif',  'only)','osf/motif', 'not)', 'life?','no?', 'all)','example)', 'card)',  '_______',  'say?','______','__________________________________________________________________________','use?','etc.),', '___/', 'tcp/ip', 'masses', 'tt', 'am.','x{','well?','they?','st.,', '_is_', 'no:', 'etc.).','___|','info?', '19,', '1/4', '1/4','26,','17,', 'pa146008@utkvm1.utk.edu', 'but...', 'pa146008@utkvm1.utk.edu',  'low.', 'hard.', '1993:', '______________________________________________________________________________', 'then?', 'above)','others)', 'above)','of)','us?','out?','question','x}','home:', 'lives,', 'it).','ca>', '_______________________________________________________________________________', 'ones,','from?', 'ed.', 'for?',"sun's", 'al.', 'gnu','saves)','_/_/_/_/',  '_/_/','jr.', 'copies)','tel#:', 'ii,', 'gl','road.','all?','fprintf(stderr,', 'check_io(output,','real','waco,',  'yours.', '________________________________________________________________________________','found','drive','key','ideas?', 'them.','--','case','makes','year','free','etc.','kind','etc.','true','told','me.','man','full','time.','that.','now,','me,','(the','this.','you.','you,','that,','this,','them,','"i','no,','here.','not.','all.','fax:','one.','up.','say,','sun','all,','there.','too.','out.','>that','(which','year.','know,','now.','(as','do.', 'and,','>this','(if', 'to:','_____','tel:','os/2','days.','etc.)','them?','own.','in,',"bd's", 'will.','n.','why?','used.','be,', 'first.','n.','down.','b)','not?','k.',"he'll",'above,','x11','a)','or,', 'man,', 'g.','for.','done.','_the','t.','at&t','do?','men,','lines:','p.s.', 'deleted]','x11r5','ed>','bad,','it."','was.','thanks!', 'of,', 'will,', 'etc.,', 'amp', 'w/','it)','be?','end,','pp.','here?', "else's",'p,#','there?', 'it!',  'i.', 'huh?','pp,', 'n3jxp','etc,', 'problem?','lines.', 'go.','see.','__/', 'm"`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@(','mind,','eh?', '____', 'lot.','ones.', 'far.','one?', 'die.','dod#','ra>','on?','_/_/_/', 'ozone', 'cnn','phi', 'geb@cadre.dsl.pitt.edu', 'so?','appears)','his/her', 'exists.', 'think)','x,',"year's",'eof_not_ok);','m/s','bds', 'm/s','god?', 'it...', 'him?', 'false.', 'public.', 'etc...','at.', 'odwyer@sse.ie', 'x/*', 'xfree86','etc)','_not_', 'to?',]
remove_list_2 = ['auth_cnt);', 'more_auths', 'oname)',  'cats............',"yorn[0]=='n'))", 'printf("\\nplease', 'eof.', 'length;', 'eatchar', 'filler', 'val', 'fputc(uuencode(val),', 'failed"', 'ioccc.', 'busy,', 'obfuscate.info', 'x11r4,', 'isc,',  '--', "that's",'apr','gmt','hm.', 'ks', 'them.)','pe-||','full....',  'janet:', 'rome,', 'ok?','huh.','article-i.d.:','you!"', 'everyone', '28', 'works','nntp-posting-host:' ,'start',  '25', 'mon,','following','17', '23', 'thu,',  'fri,','tue,','gmt', 'enough','christian@aramis.rutgers.edu','distribution:', 'reply-to:', 'thu','certain','date:', 'message-id:', 'sender:', '21', 'organization:', 'references:', 'nntp-posting-host:''known', 'write', 'consider','april','26','15', 'understand','30', 'full',  'ask', 'pay', 'feel',  'went', 'mail','found','13','simply',   'says', '27', 'kind','send', 'far', 'whether', 'quite', 'work', 'article','several','32','gets','>this','up.', 'them,', "what's",'$100','-->','2)','saw', 'this?', '54','6)',  '(415)', '--if', 'it...', "it'll", '81','whoever', '>well,', '>who', '>>>>','8,', '20,','hopefully', '44',"'92",  '>>the','72',"'91",'it)', 'thoughts', '>which',   '---------','$1', 'away','2.','3.', 'so.', 'mentioned', '(as', 'j.','#>','whose', 'him.','14','11','took','supposed','too.', 'p.','say,', 'too.', 'p.', '12','24','cause','20','within','similar', 'instead', 'via',  'everything','often','well,', 'that.', 'lines:', 'me,','though','now,','making',  'came', 'perhaps', 'comes',  'means', 'based',  'heard','ever','seen','given', 'seem', 'always', 'three', 'maybe', 'less','next', 'new','nothing', 'great','>were', 'you!', '>>for', '(c)', 'thanx','=============================================================================','however.' 'article','else', 'available','copy', 'that,',  'makes','use.', '(--)',  'yes.','\\/', 'k',  '29', '1st','------------------------------------------------------------------------------',  'do.', 'knew', '>if',':)','>it', 'yet', 'remember', 'trying','93', '4.','16', '9', 'db','>as', '__', 'yes',  '>--','"i','said,', 'clear', 'difference', 'asked', 'especially', 'four', 'upon','lot', 'able', 'keep','least','let',  'long', 'tell','help','look', 'problem', 'however,', 'best', 'thing', 'rather', 'first', 'since', 'really', 'believe', 'anyone', 'right', 'something','bd','people','much','part','every', 'little', 'used', 'need', 'good','no.','***', 'on,','//', '80','----------','(a)','58', 'u.', 'co', '>>is', 'cdt@vos.stratus.com', '*****','--------------------------------------------------------------------------', '73', '_/_/_/',  'm"`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@("`@(','>their','*******************************************************************************', 'pa', '(the',  '_/','/*', "there's","they're", 'also,', 'gave','>that', '>a', 'cut', 'not,', '1)','>what','---------------------------------------------------------------------', 'bbs', 'w.', '3)', '>>in', 'oh', 'm.', 'joe', 'became', '18','still', 'around', 'it,','although','actually','them.','almost', 'show', 'agree', 'certainly','anything', 'try','give', 'put', 'seems', 'probably','might','would','made', 'well', 'got', 'another', 'better', 'number', 'read', 'come', 'sure', 'things', 'someone','never', 'please', 'back', 'last', 'without','going', 'find', 'using','64','*not*',"he's",'want','way', 'say','>of','it?', '??','to.','----------------------------------------------------------------------------','-------------------------------------------------------------------------------', 'hp', 'lets', '55', '$3','etc.','|>|>', 'fine,', '"i\'m','>and','you.', 'you.','saying','like','8', 'whole', '6', 'told','make', 'is,','many','either', '1993','go', 'us','see','said', 'take','q','???','>:',  '>>>','six', 'r','>is','19', '22', '55.0', 'dr.', 'all,', '**','||', '(in','----','----------------------------------------------------------------------','$50','>|>',"i'll",  'you,', '>to','end', 'this.', '*/','(or',  '(i','"the','big', 'evidence', 'getting', 'day', 'thought', 'car', 'done', 'cannot', 'high', 'order', 'person', '10', 'name', '"the''----------------------------------------------------------------------','!=','--------------------------------------------------------------------------------', '7', '---',':-)','(and',':-)', ':-)',':-)' 'is,', 'one', '|>','5', '...', '>in','must', "i've",'>the','4', '>i',"i'd",  '3', "can't",'get', '>>', 'know', 'x','two', '0', 'it.', 'time', '2', "max>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'", 'think', 'writes:', 'use', 'also', 'could', 'even', '1', 'may', "i'm",]
# print(len(remove_list_1), len(remove_list_2))

# importing regex library to apply regex in features.
import re 
print(len(features),end=" ")
for x in features:
    if x in remove_list_1 or x in remove_list_2:
        features.remove(x)
print(len(features))
# applying some regex

exp = re.compile('\W')
exp2 = re.compile('\d*\W')
exp3 = re.compile('^[\W_]+$')

print(len(features),end=" ")
features = [x for x in features if not exp.match(x)]

print(len(features),end=" ")
features = [x for x in features if not exp2.match(x)]

print(len(features),end=" ")
features = [x for x in features if not exp3.match(x)]
print(len(features))

424 452
183042 182531
182531 147944 131335 131246


In [ ]:
# this is the list of all words(stored in desc order of their count) which can be made as an feature.
print(features,end="")

In [17]:
# applying regex to remove digits.
exp = re.compile('\d')
features = [x for x in features if not exp.match(x)]
len(features)

129479

In [ ]:
# taking features from 30 to 7000.(top_freq word).
# new_features is the actual words i'm using in this code as features.

new_features = features[15:700]
print(len(new_features))
print(new_features,end="")    # list of words i'm using as features.

Now below is the code for self implementing Multinomial Naive Bayes and I've also done the predictions using Sklearn MultiNomialNB for comparing the results.

In [19]:
# fit func will fit our data into dictionary according to target values.
                                        #(structure of dict.): 
                             
                             # dictionary['target_name']['feature_name']['count_feature'].
            
    
def fit(training_path, target, features):        # defining fit func.
    new_dictionary = {}                          # initialising dictionary
    
    for name in target:               # iteratng over each target values.
        path = training_path + name   
    #dic[name]['total_count']=0
        val = 0                        # initialising temp variables.
        ct = 0
        temp_arr = np.zeros((len(features),1))             # this numpy_array will store count of each features in particular folder.
        
        for file in glob.glob(os.path.join(path, '*')):    # iterating over each file in target value.
            file_o = open(file,'r').read()                 # for removing header.
            new_list = file_o.split()
            i=0
            while new_list[i]!='Date' and new_list[i]!='date' and new_list[i]!='Date:':
                i+=1
            k_cnt = 0.03*(len(new_list))                    # removing starting 3% words from each file after 'date' encountered.
            k_cnt = int(k_cnt)
            new_list = new_list[k+i-8:]
            new_list = [x.lower() for x in new_list]       # changing all words into lower_case.
            new_dictionary[name] = {}                      # initialising dict with that particular target value.
            for i in range(len(features)):
                temp_arr[i][0] += new_list.count(features[i])    # finding count of each feature in each file of target folder.
            val+=len(new_list)             
            ct+=1                                           # ct will store count of each file present in particular target folder. 
            
        # storing count of each features in each folder in dict.
        for i in range(len(features)):
            new_dictionary[name][features[i]] = temp_arr[i][0]
            
        # 'val' will contains count of all_words present in particular target folder.
        new_dictionary[name]["count_words_inFolder"] = val         # total count of words in particular target folder
        new_dictionary[name]['count_currClass'] = ct               # count of all files in particular folder i.e output m kitni bar woh target aara ha.
   
    tot_cnt = 0              # this will store all the counts of all target values i.e len(output).
    for name in target:
        tot_cnt += new_dictionary[name]['count_currClass']
    new_dictionary['all_count'] = tot_cnt
    
    return new_dictionary

In [20]:
# this function will cal the probability required to predict the class.

def cal_probability(new_list, dictionary, curr_class):
    output = 0                              # initialising output as 0 (using log)
    features = dictionary[curr_class].keys()
    features = list(features)                       # extracting total features.
    for i in range(len(features)):                  # loop over all possible features.
        if(features[i] == 'count_currClass' or features[i] == 'count_words_inFolder'): # some extra variables other then features, so continue.
            continue
            
        count_part_feature = new_list.count(features[i])             # counting the occurence of a particular feature in test file.
        count_feature_dict = dictionary[curr_class][features[i]] + 1    # occurence of part. feature in from dictionary
        count_words_currClass = dictionary[curr_class]['count_words_inFolder'] + (len(features)-2)   # total words in current_class.
        
        # p will store: (probability that a particular feature belongs to curr_class) ** (occurence of that feature in test_file).
        p = ((np.log(count_feature_dict) - np.log(count_words_currClass)) * count_part_feature)
        output = output + p
    
    #print("prob:", output)
    count_class = dictionary[curr_class]['count_currClass']  # count of that particular class.(cnt of part. output)
    total_count = dictionary['all_count']                    # total class count.(total output)
    class_prob = np.log(count_class) - np.log(total_count)   # prob. of particular class.
    output = output + class_prob
    
    return output                          # return calculated prob.

In [21]:
# function to predict for a single_file.
def  Predict_for_singleFile(new_list, dictionary):
    classes = dictionary.keys()        # extracting all possile outputs.
    max_prob = -1                      # init. max_prob as -1
    first_iter = False
    prob = 0                           # this will give curr_calculated_prob.
    best_class = -1
    for curr_class in classes:         # for outpur in all poss_output:
        if(curr_class == 'all_count'):
            continue
        prob = cal_probability(new_list, dictionary, curr_class)     # cal_prob for a single file belong to curr_class
        #print(prob)
        if(prob > max_prob or first_iter==False):                    # if calculated_prob > max_prob thn update it and update best_class.
            max_prob = prob
            best_class = curr_class
            first_iter = True
    return best_class                    # return best_class

In [22]:
# function to predict output.
def predict(path_x_test, folder_name_list, dictionary):
    y_pred = []    # init. y_pred as NULL
    temp = []
    for name in folder_name_list:                   # for output in all poss_output.
        #print("name:", name)
        path = path_x_test + name 
        cnt1=0
        cnt2=0
        #temp_dict[name]={}
        for file in glob.glob(os.path.join(path, '*')):    # for each file in particular folder.
            file_o = open(file,'r').read()[400:]
            new_list = file_o.split()
            output = Predict_for_singleFile(new_list, dictionary)         # finding best_class for that file.
            if(cnt2%40 == 0):
                print(output,end=" ")
            cnt2+=1
            if(output==name):
                cnt1+=1
            y_pred.append(output)                          # adding the predicted best_class to y_pred.
        print("count:", cnt1)
        temp.append((cnt1,cnt2))
    return y_pred, temp               # return y_pred.

In [23]:
# Accuracy Function
def get_score(prediction):
    sum_p = 0                 # sum_p is predicted sum
    sum_a = 0                 # sum_a is actual sum(sum of no of files in each folder.)
    for p,a in prediction:    # 'p' is predicted count and 'a' is actual count
        sum_p += p
        sum_a += a
    return ((sum_p/sum_a)*100)  # return accuracy.

# print_outpt
def prt_output(prediction):
    for p,a in prediction:
        print("Correctly Predicted:", p, "Total:", a, "Classes")

In [43]:
# UNCOMMENT THIS TO RUN PREDICT FUNCTION.
# set path where testing data is stored.

# path_test = "C:\\Users\\Archit\\Desktop\\new\\20_newsgroups_test\\"
# y_pred, temp = predict(path_test, target, new_dictionary)

In [26]:
# printing number of correct_prediction and accuracy.
prt_output(temp)
get_score(temp)

Correctly Predicted: 127 Total: 200 Classes
Correctly Predicted: 141 Total: 200 Classes
Correctly Predicted: 126 Total: 200 Classes
Correctly Predicted: 143 Total: 200 Classes
Correctly Predicted: 141 Total: 200 Classes
Correctly Predicted: 132 Total: 200 Classes
Correctly Predicted: 148 Total: 200 Classes
Correctly Predicted: 167 Total: 200 Classes
Correctly Predicted: 149 Total: 200 Classes
Correctly Predicted: 166 Total: 200 Classes
Correctly Predicted: 132 Total: 200 Classes
Correctly Predicted: 160 Total: 200 Classes
Correctly Predicted: 134 Total: 200 Classes
Correctly Predicted: 145 Total: 200 Classes
Correctly Predicted: 141 Total: 200 Classes
Correctly Predicted: 152 Total: 200 Classes
Correctly Predicted: 142 Total: 200 Classes
Correctly Predicted: 142 Total: 200 Classes
Correctly Predicted: 129 Total: 200 Classes
Correctly Predicted: 93 Total: 200 Classes


70.25

Now below is the code for using sklearn implemented MultiNomialNB.
First I've created the DataFrame to visualise the data for training and testing.

In [27]:
# process starts for creating data_frame 
data_frame_features = new_features

In [28]:
len(data_frame_features)

6985

In [ ]:
                                    # DATA_FRAME creation.(for training files.)
# creating data-frame.
k1=0        # init k1.
data_frame_features.append('target_values_new')             # appending extra column in features to store the output.
df_new = pd.DataFrame(columns = data_frame_features)        # creating an empty dataframe with columns as features and one extra column as output.

for new_name in target:                       # iterating over each folder:
    print("name:", new_name)
    #print()
    path = "C:\\Users\\Archit\\Desktop\\new\\20_newsgroups_train\\" + new_name
    for new_file in glob.glob(os.path.join(path, '*')):     # iterating over each file in that particular folder.
       # print("file: ",new_file)
        cnt_array = np.zeros((len(data_frame_features),1))   # creating an np array to store count of each 'feature' in each file.
        file_n = open(new_file,'r').read()
        new_words = file_n.split()                           # 'new_words' is a list that stores each word of a file.
        new_words = [x.lower() for x in new_words]
        
        for f in range(0,len(data_frame_features)):            # iterating over each feature.
            
            count_w=0                                          # count_w will store count of f'th feature in that file.
            if(data_frame_features[f]=='target_values_new'):
                continue;
            elif data_frame_features[f] not in new_words:      # if feature is not present in file continue(as np array is init as 0).
                continue;
            for w in new_words:
                if w == data_frame_features[f]:
                    count_w+=1;
            cnt_array[f][0] = count_w                          # stores count of that feature

        df_new.loc[k1] = [cnt_array[i][0] for i in range(cnt_array.shape[0])]      # appending that np.array as a row in our data_frame.
        df_new.loc[k1, 'target_values_new'] = new_name          # adding target value for that particular row.
        k1+=1

print("DataFrame Created.")

In [30]:
df_new.head(5)
# About DataFrame:
# Each row represents a particular email and the last col in each row represents the class it belongs to.
# Each col is a word(top frequency word) from the whole dataset.
# DF(i, j) represents the count of j'th word in i'th email.

,article,any,some,who,no,so,which,when,don't,were,...,"island,","cover,",houses.,"tradition,",interprets,jul,hash,masks,urban,target_values_new
0,0.0,2.0,3.0,3.0,0.0,3.0,8.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,alt.atheism
1,3.0,15.0,31.0,23.0,22.0,25.0,14.0,12.0,15.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,alt.atheism
2,1.0,2.0,2.0,0.0,2.0,1.0,1.0,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,alt.atheism
3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,alt.atheism
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,alt.atheism


In [32]:
# re-initialising 'data_frame_features' to new_features.
while 'target_values_new' in new_features:
    new_features.remove('target_values_new')
data_frame_features = new_features
print(len(data_frame_features))

6985


In [33]:
                                            # DATA_FRAME creation(for testing file.)
# creating data-frame.
k1=0        # init k1.
data_frame_features.append('target_values_new')             # appending exttra column in features to store the output.
test_df = pd.DataFrame(columns = data_frame_features)        # creating an empty dataframe with columns as features and one extra column as output.

for new_name in target:                       # iterating over each folder:
    print(new_name,end=" ")
    
    path = "C:\\Users\\Archit\\Desktop\\new\\20_newsgroups_test\\" + new_name    # path of testing_file.
    
    for new_file in glob.glob(os.path.join(path, '*')):     # iterating over each file in that particular folder.
        cnt_array = np.zeros((len(data_frame_features),1))   # creating an np array to store count of each 'feature' in each file.
        file_n = open(new_file,'r').read()
        new_words = file_n.split()                           # 'new_words' is a list that stores each word of a file.
        new_words = [x.lower() for x in new_words]
        
        for f in range(0,len(data_frame_features)):            # iterating over each feature.
            count_w=0                                          # 'count_w' will store count of f'th feature in that file.
            if(data_frame_features[f]=='target_values_new'):
                continue;
            elif data_frame_features[f] not in new_words:      # if feature is not present in file continue(as np array is init as 0).
                continue;
            for w in new_words:
                if w == data_frame_features[f]:                # calculating occur. of that part. feature in test_file.
                    count_w+=1;
            cnt_array[f][0] = count_w                          # stores count of that feature

        test_df.loc[k1] = [cnt_array[i][0] for i in range(cnt_array.shape[0])]      # appending that np.array as a row in our data_frame.
        test_df.loc[k1, 'target_values_new'] = new_name                    # adding target value for that particular row.
        k1+=1

print()
print("Test DataFrame Created.")

alt.atheism comp.graphics comp.os.ms-windows.misc comp.sys.ibm.pc.hardware comp.sys.mac.hardware comp.windows.x misc.forsale rec.autos rec.motorcycles rec.sport.baseball rec.sport.hockey sci.crypt sci.electronics sci.med sci.space soc.religion.christian talk.politics.guns talk.politics.mideast talk.politics.misc talk.religion.misc 
Test DataFrame Created.


In [35]:
# splitting training_data into input and output.
x_train = df_new.values[:, :-1]
y_train = df_new.values[:, -1]
x_train.shape, y_train.shape

((15997, 6985), (15997,))

In [37]:
# splitting testing_data into i/o and o/p.
x_test = test_df.values[:, :-1]
y_test = test_df.values[:, -1]
x_test.shape, y_test.shape

((4000, 6985), (4000,))

In [38]:
# Importing required libraries:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import time

In [39]:
# clf is the classifier of MultiNomialNB(MNB).
st = time.time()
clf = MultinomialNB()
clf.fit(x_train,y_train)
et = time.time()
y_pred = clf.predict(x_test)
print(y_pred.shape)
time_taken = et-st
print(time_taken)

(4000,)
41.39916014671326


In [40]:
clf.score(x_test,y_test) #nice.

0.79925

In [41]:
print(classification_report(y_test, y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.64      0.77      0.70       200
           comp.graphics       0.69      0.79      0.74       200
 comp.os.ms-windows.misc       0.64      0.72      0.68       200
comp.sys.ibm.pc.hardware       0.75      0.74      0.74       200
   comp.sys.mac.hardware       0.80      0.81      0.80       200
          comp.windows.x       0.88      0.64      0.74       200
            misc.forsale       0.78      0.89      0.83       200
               rec.autos       0.85      0.95      0.90       200
         rec.motorcycles       0.84      0.94      0.89       200
      rec.sport.baseball       0.96      0.97      0.96       200
        rec.sport.hockey       0.96      0.95      0.96       200
               sci.crypt       0.92      0.88      0.90       200
         sci.electronics       0.80      0.78      0.79       200
                 sci.med       0.89      0.82      0.86       200
         

In [42]:
print(confusion_matrix(y_test, y_pred))

[[153   1   1   0   0   0   1   0   2   0   0   0   1   2   1   3   1   2
    0  32]
 [  2 158   8   5   7   4   1   0   1   0   1   3   3   2   0   1   0   0
    0   4]
 [  0  14 144  14   7   8   1   1   2   1   0   1   2   0   0   0   1   0
    2   2]
 [  0   4  13 148  14   1  10   1   0   0   0   0   8   0   1   0   0   0
    0   0]
 [  0   2   8  13 162   0   8   2   0   0   0   1   2   0   2   0   0   0
    0   0]
 [  0  22  40   2   1 127   3   0   2   0   1   0   2   0   0   0   0   0
    0   0]
 [  0   1   0  10   1   0 178   2   2   1   0   0   3   0   0   1   0   0
    1   0]
 [  0   1   1   0   0   0   4 190   0   0   0   1   0   0   0   0   3   0
    0   0]
 [  0   1   0   0   0   0   4   4 188   0   0   0   2   0   0   0   0   0
    1   0]
 [  0   0   0   0   0   0   2   1   0 194   2   0   0   0   0   0   0   0
    1   0]
 [  1   0   0   0   0   0   2   0   1   1 191   0   0   1   0   1   0   0
    2   0]
 [  0   6   2   1   3   1   1   1   1   0   0 176   4   0   1   0